3D SLAM with Lie Group SE(3), Barfoot.

Transformation Matrix 

At time k,
 
$T_{k} = \begin{bmatrix} {}^{i}C^{v_{k}} &  - {}^{i}C^{v_{k}} {}^{i}r^{v_{k}} \\ 0_{1x3} & 1 \end{bmatrix} \in SE(3)$

where <br>
$^{i}C^{v_{k}}$ is rotation matrix from inertial frame to vehicl (body) frame at time k <br>
$^{i}r^{v_{k}}$ is translation vector from origin of inertial frame to the orgin point of vehicle frame at time k


Error Cost Function

At time k,

$e_{k} = \sum_{j} (y_{k} - T_{k} * p_{j})$

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import casadi as ca
import BF_PCA
import time

import sys
sys.path.insert(0, '../')
from pyecca.lie import se3, so3, matrix_lie_group

from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from matplotlib.text import Annotation

In [2]:
class Arrow3D(FancyArrowPatch):

    def __init__(self, x, y, z, dx, dy, dz, *args, **kwargs):
        super().__init__((0, 0), (0, 0), *args, **kwargs)
        self._xyz = (x, y, z)
        self._dxdydz = (dx, dy, dz)

    def draw(self, renderer):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        super().draw(renderer)
        
    def do_3d_projection(self, renderer=None):
        x1, y1, z1 = self._xyz
        dx, dy, dz = self._dxdydz
        x2, y2, z2 = (x1 + dx, y1 + dy, z1 + dz)

        xs, ys, zs = proj_transform((x1, x2), (y1, y2), (z1, z2), self.axes.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))

        return np.min(zs) 
    
def _arrow3D(ax, x, y, z, dx, dy, dz, *args, **kwargs):

    arrow = Arrow3D(x, y, z, dx, dy, dz, *args, **kwargs)
    ax.add_artist(arrow)

setattr(Axes3D, 'arrow3D', _arrow3D)

class Annotation3D(Annotation):

    def __init__(self, text, xyz, *args, **kwargs):
        super().__init__(text, xy=(0, 0), *args, **kwargs)
        self._xyz = xyz

    def draw(self, renderer):
        x2, y2, z2 = proj_transform(*self._xyz, self.axes.M)
        self.xy = (x2, y2)
        super().draw(renderer)



def _annotate3D(ax, text, xyz, *args, **kwargs):
    '''Add anotation `text` to an `Axes3d` instance.'''

    annotation = Annotation3D(text, xyz, *args, **kwargs)
    ax.add_artist(annotation)

setattr(Axes3D, 'annotate3D', _annotate3D)

In [3]:
t0 = time.time()

#------------------------
# Define constant

deg_to_rad = np.pi / 180

noise={
    'odom_std': 0.0,
    'x_std': 0.0,
    'y_std': 0.0,
    'z_std': 0.0,
    'odom_bx_bias': 0.0,
    'odom_by_bias': 0.0,
    'odom_bz_bias': 0.0
}

#------------------------
# Initialize states

#input pair: [dist , yaw                ,  pitch              ,  roll]
u = np.array([[3   , 0                  , 0                   ,    0],
             [5    , 53 * deg_to_rad    , 0      ,              0],
             [1    , 20 * deg_to_rad    , 0                   ,    0],
             [2    , 17 * deg_to_rad    , 0                   ,    0],
             [4    , 20 * deg_to_rad    , 0                   ,    0],])
             # [5    , 0 * deg_to_rad    , 0                   ,    0],
             # [3    , 60 * deg_to_rad    , 0                   ,    0],
             # [2    , 20 * deg_to_rad    , 0                   ,    0],
             # [4    , 30 * deg_to_rad    , 0                   ,    0],
             # [3    , 30 * deg_to_rad    , 0                   ,    0],])

#Time array 
dt = 1
tf = len(u) * dt

#Define landmarks and coresponding weight
landmarks = np.array([[1,1,0],
                     [3,4,0],
                     [6,5,0],
                     [3,8,0],
                     [2,6,0]])
weight = np.array([1,
                   1,
                   1,
                   1,
                   1])

#Define true state variable x described in inertial frame
x0 = np.array([0.,0.,0.]);
x_prev = x0;
Rot0 = np.identity(3)

#Define estimated state wrt inetial frame
xh = np.array([x0])
Roth = Rot0
Rotk = Rot0

SE3 = se3._SE3()

J = ca.SX.zeros(1,1)

hist = {'x': [x0],
        'y': [],
        'R':  [Rot0],
        'T': [],
        'assoc':[],
       'Frame': ['inertial']};
Top_allbody = []
yk_allbody = []
assoc_allbody = []


#------------------------
#Begin Simulation

t_vect = np.arange(0,tf,dt)
for k, tk in enumerate(t_vect):
    
    # print("iteration",k)
    
    # !!! Move then measure !!!
    
    #Assign Rotation and translation Input
    # !!! rotate then translate !!!
    uk = u[k]
    
    yawk = uk[1]
    pitchk = uk[2]
    rollk = uk[3]
    
    Rot_prev2body   = BF_PCA.euler2rot(yawk, pitchk, rollk) #previous body to current body
    Rotk = Rotk@Rot_prev2body #inertial to current body
    
    uTrans_k = Rotk@np.array([uk[0], 0, 0]).T
    
    #Update State variables (x is in inertial frame) xk = r_vk_i (barfoot)
    xk = x_prev + uTrans_k; 
    
    #Measure landmarks
    yk = BF_PCA.measure_landmark(Rotk, xk, landmarks, noise=None, range_max=1000)
    yk_allbody.append(yk)
    
    #Transformation Matrix (True)
    Tk = np.vstack([np.hstack([Rotk.T, np.matmul(-Rotk.T,xk).reshape(3,1)]), np.array([[0, 0, 0, 1]])])
    
    #Define symbolic perturbation of T
    trans_sym = ca.vertcat(ca.SX.sym('u_'+str(k)), ca.SX.sym('v_'+str(k)), ca.SX.sym('w_'+str(k)))
    angles_sym = ca.vertcat(ca.SX.sym('omega1_'+str(k)), ca.SX.sym('omega2_'+str(k)), ca.SX.sym('omega3_'+str(k)))
    epsilon = ca.vertcat(trans_sym, angles_sym)
    if k == 0:
        epsilon_all = epsilon
    else:
        epsilon_all = ca.vertcat(epsilon_all, epsilon)

    #Associate landmarks index to measurement observed using maximum likelihood
    assoc = [ BF_PCA.data_association(xk, yi, landmarks, Rotk) for yi in yk ]
    assoc_allbody.append(assoc)
    
    #Re-assign previous state
    x_prev = xk
    
    # Initialize Operating Point T
    That = SE3.exp(SE3.wedge(np.array([1000,1,0,10,10,10])))
    if k == 0:
        Top = SE3.exp(SE3.wedge(np.array([0,0,0,0,0,0])))
    
    #----- Point Cloud Alignment, iterative optimization for each time step k -------
    
    while (ca.norm_fro(That[:3,:3]-Top[:3,:3])+ca.norm_2(That[:3,3]-Top[:3,3]))>10**-4:
        Top = That
        
        #cost function (Barfoot 8.95)
        J = BF_PCA.build_cost_barfoot(Top=Top , 
                                      p=landmarks, 
                                      y=yk, 
                                      assoc=assoc,
                                      weight=weight,
                                      epsilon=epsilon)
        
        # Symbols/expressions
        nlp = {}                 # NLP declaration
        nlp['x'] = epsilon       # decision vars
        nlp['f'] = J             # objective
        nlp['g'] = 0             # constraints
        
        # Create solver instance
        opts = {'ipopt.print_level':0, 'print_time':0, 'ipopt.sb': 'yes'}
        F = ca.nlpsol('F','ipopt',nlp, opts);
        
        # Solve the problem using a guess
        # This uses original landmark/measure association (associates which landmark we think the measurement is measuring)
        x_input = np.zeros(6)
        optim = F(x0=x_input)
        
        That = SE3.exp(SE3.wedge(optim['x']))@Top
        
    #--------------------------------------
    
    Top = That
    Top_allbody.append(Top)
    
    
    
#     #----- Point Cloud Tracking, optimization for all time -------
#     J = ca.SX.zeros(1,1)
#     for i in range(k+1):
#         #Update cost function (Barfoot 8.95)
#         J += BF_PCA.build_cost_barfoot(Top=Top_allbody[i] , 
#                                        p=landmarks, 
#                                        y=yk_allbody[i], 
#                                        assoc=assoc_allbody[i], 
#                                        epsilon=epsilon)


    
#     # Non-linear programming
#     #   for point cloud alignment problem, we are solving for "T" only which include of:
#     # - rotation matrix from inertial-frame to vehicle-frame
#     # - translation vector describing the vehicle's pose in inertial-frame

#     # Symbols/expressions
#     nlp = {}                 # NLP declaration
#     nlp['x'] = epsilon_all   # decision vars
#     nlp['f'] = J             # objective
#     nlp['g'] = 0             # constraints

#     # Create solver instance
#     opts = {'ipopt.print_level':0, 'print_time':0, 'ipopt.sb': 'yes'}
#     F = ca.nlpsol('F','ipopt',nlp, opts);
    
#     # Solve the problem using a guess
#     # This uses original landmark/measure association (associates which landmark we think the measurement is measuring)
#     eps = 1e-20
#     x_input = np.ones(6*(k+1)) * eps
#     optim = F(x0=x_input)

#     for i in range(k+1):
#         That = SE3.exp(SE3.wedge(optim['x'][(i*6):(i*6)+6]))@Top_allbody[i]
#         Top_allbody[i] = That
        
#     #------------------------------------------------

    #Append "true state" history
    hist['x'].append(xk)
    hist['R'].append(Rotk)
    hist['T'].append(Tk)
    hist['Frame'].append(k)
    for yi in yk:
        hist['y'].append(np.hstack([yi, k]))      # History of measurements recorded at each time step
    for i in range(len(assoc)):
        hist['assoc'].append(np.hstack(['p'+str(assoc[i]), 'x'+str(k)]))
 
    
for key in hist.keys():
    hist[key] = np.array(hist[key])

tf =  time.time() - t0
print('runtime [s]: ', tf)
# hist

runtime [s]:  0.6225972175598145


In [5]:
print(Top_allbody[1])
print(hist['T'][1])

@1=0, 
[[0.601815, 0.798636, -2.18365e-17, -6.80545], 
 [-0.798636, 0.601815, 2.16564e-16, 2.39591], 
 [-2.52654e-23, -5.9292e-24, 1, 5.77306e-20], 
 [@1, @1, @1, 1]]
[[ 0.60181502  0.79863551  0.         -6.80544507]
 [-0.79863551  0.60181502  0.          2.39590653]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


In [12]:
# 3D plot
fig = plt.figure(1)
ax = fig.add_subplot(111, projection='3d')

#Plot true state
ax.plot(hist['x'][:, 0] , hist['x'][: , 1], hist['x'][:, 2], label="True State")
ax.set_title("Point Cloud Alignment")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

#Plot body-frames (arrow is assumed to be equivalent to x-axis of the body frame)
textEnable = True
Rotate_x2y = BF_PCA.euler2rot(np.pi/2, 0 ,0)
Rotate_x2z = BF_PCA.euler2rot(0, -np.pi/2, 0)

x = hist['x']
Rot = hist['R']
for i in range(len(x)):
    if textEnable:
        ax.annotate3D(hist['Frame'][i], x[i],
              xytext=x[i,:2]+[-1,-1],
              textcoords='offset points',
              bbox=dict(boxstyle="round", fc="lightyellow"))

    
    x_axis = np.matmul(Rot[i] , [1, 0 , 0])
    y_axis = Rotate_x2y@np.matmul(Rot[i] , [1 , 0 , 0])
    # z_axis = Rotate_x2z@np.matmul(Rot[i] , [1 , 0, 0])
    ax.arrow3D(x[i,0] , x[i, 1] , x[i, 2], x_axis[0] , x_axis[1], x_axis[2],mutation_scale=10, ec ='red',fc='red')
    ax.arrow3D(x[i,0] , x[i, 1] , x[i, 2], y_axis[0] , y_axis[1], y_axis[2],mutation_scale=10, ec ='green',fc='green')
    # ax.arrow3D(x[i,0] , x[i, 1] , x[i, 2], z_axis[0] , z_axis[1], z_axis[2],mutation_scale=10, ec ='blue',fc='blue')

#Plot Landmarks
for m in landmarks:
    ax.scatter(m[0],m[1],m[2],s=30, marker='o', color='black')

#Plot measurement
# for arr in hist['y']:
#     zbody = arr[:3]
#     i  = int(arr[-1])
#     zinertial = np.matmul(Rot[i+1] , zbody)
#     ax.arrow3D(x[i + 1, 0] , x[i + 1, 1] , x[i + 1, 2], zinertial[0] , zinertial[1], zinertial[2],mutation_scale=10, ec ='black',fc='black')

ax.legend()

In [13]:
# 2D plot
fig = plt.figure(2)

#Plot true vs. estimated state
plt.plot(hist['x'][:, 0] , hist['x'][: , 1], label="True State")
plt.title("Point Cloud Alignment (2D Projection)")
plt.xlabel('x')
plt.ylabel('y')

textEnable = True
Rotate_x2y = BF_PCA.euler2rot(np.pi/2, 0 ,0)

#Plot body-frames (arrow is assumed to be equivalent to x-axis of the body frame)
x = hist['x']
Rot = hist['R']
for i in range(len(x)):
    
    if textEnable:
        plt.text(x[i,0] + 0.15 , x[i, 1] + 0.15, hist['Frame'][i])
    
    x_axis = np.matmul(Rot[i] , [0.5 , 0 , 0])
    y_axis = Rotate_x2y@np.matmul(Rot[i] , [0.5 , 0 , 0])
    plt.arrow(x[i,0] , x[i, 1] , x_axis[0] , x_axis[1], width=0.01, length_includes_head=True, head_width=0.1,head_length=0.1)
    plt.arrow(x[i,0] , x[i, 1] , y_axis[0] , y_axis[1], width=0.01, length_includes_head=True, head_width=0.1,head_length=0.1)
    
#Plot Landmarks
for m in landmarks:
    plt.plot(m[0],m[1],'ro')

    
#Plot measurement
# for arr in hist['y']:
#     zi = arr[:3]
#     i  = int(arr[-1])
#     plt.arrow(hist['x'][i + 1, 0] , hist['x'][i + 1, 1], np.matmul(Rot[i+1] , zi)[0] ,np.matmul(Rot[i+1] , zi)[1], width=0.06, head_width=0.1, head_length=0.1, length_includes_head=True)
    
plt.legend()